In [2]:
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="best_50_v10_float16.tflite")
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load class labels (adjust based on your model)
class_labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]  # Replace with your actual class names

# Function to preprocess the image for inference
def preprocess_image(image_path, input_shape):
    """
    Preprocess the image to match the model's input requirements.
    Args:
        image_path: Path to the input image.
        input_shape: Shape of the model's input tensor.
    Returns:
        Preprocessed image as a numpy array.
    """
    image = Image.open(image_path).convert("RGB")  # Ensure RGB format
    image = image.resize((input_shape[1], input_shape[2]))  # Resize to model input size
    image_array = np.array(image, dtype=np.float32) / 255.0  # Normalize pixel values
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    return image_array

# Function to post-process inference results
def postprocess_output(output_data, original_image_shape):
    """
    Post-process the raw output data from the model.
    Args:
        output_data: Raw output data from the TensorFlow Lite model.
        original_image_shape: Shape of the original input image (height, width).
    Returns:
        Bounding boxes in [xmin, ymin, xmax, ymax] format, class names, and confidence scores.
    """
    boxes = []  # Bounding box coordinates
    class_names = []  # Predicted class names
    confidences = []  # Confidence scores
    
    # Example decoding logic (adjust based on your model's output format)
    for detection in output_data[0]:  # Assuming output_data[0] contains bounding boxes
        confidence = detection[4]  # Confidence score (adjust index based on your model)
        if confidence > 0.5:  # Threshold for valid detections
            x_min, y_min, x_max, y_max = detection[:4] * np.array(
                [original_image_shape[1], original_image_shape[0], original_image_shape[1], original_image_shape[0]]
            )
            boxes.append([x_min, y_min, x_max, y_max])
            class_id = int(detection[5])  # Class ID (adjust index based on your model)
            class_names.append(class_labels[class_id])  # Map ID to class name
            confidences.append(confidence)
    
    return boxes, class_names, confidences

# Function to draw bounding boxes on an image
def draw_bounding_boxes(image, boxes, class_names=None, confidences=None, color=(0, 255, 0), thickness=2):
    """
    Draw bounding boxes on an image.
    Args:
        image: Input image (numpy array).
        boxes: List of bounding boxes in [xmin, ymin, xmax, ymax] format.
        class_names: List of predicted class names corresponding to each box.
        confidences: List of confidence scores corresponding to each box.
        color: Color of the bounding box (default is green).
        thickness: Thickness of the box lines (default is 2).
    Returns:
        Annotated image with bounding boxes.
    """
    for i, box in enumerate(boxes):
        # Draw rectangle
        cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, thickness)
        
        # Add label and confidence if provided
        if class_names and confidences:
            label = f"{class_names[i]} ({confidences[i]:.2f})"
            cv2.putText(image, label, (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)
    
    return image

# Path to your input image
image_path = "images/10.png"  # Replace with your actual image path

# Load and preprocess the image for inference
input_shape = input_details[0]['shape']
input_data = preprocess_image(image_path, input_shape)

# Perform inference
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get raw output data from the model
output_data = interpreter.get_tensor(output_details[0]['index'])

# Post-process output data to extract bounding boxes and labels
original_image = cv2.imread(image_path)  # Load original image with OpenCV for drawing
original_image_shape = original_image.shape[:2]  # Get height and width of original image

bounding_boxes, predicted_classes, confidence_scores = postprocess_output(output_data, original_image_shape)

# Draw bounding boxes on the original image
annotated_image = draw_bounding_boxes(original_image.copy(), bounding_boxes, predicted_classes, confidence_scores)

# Display and save the annotated image


cv2.imwrite("output_image.jpg", annotated_image)  # Save annotated image as a file


/Users/silunikeerthiratne/Documents/IoT/Code/myenvIot/lib/python3.11/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


True

In [3]:
print(bounding_boxes)
print(predicted_classes)
print(confidence_scores)

[[1737.8199815750122, 617.6650822162628, 2205.705189704895, 899.7697234153748], [610.192745923996, 600.66819190979, 1080.33447265625, 887.7187371253967]]
['8', '8']
[0.68512034, 0.59686565]
